In [1]:
import os
import numpy as np
import pandas as pd
import torch
import random
import re

from tqdm import tqdm
from itertools import combinations
from collections import deque
from transformers import AutoTokenizer
from rank_bm25 import BM25Okapi
from sklearn.model_selection import train_test_split

/root/anaconda3/envs/code_sim/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def seed_everything(seed=42):
  random.seed(seed)
  np.random.seed(seed)
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False
  torch.manual_seed(seed)
  torch.cuda.manual_seed(seed)
  torch.cuda.manual_seed_all(seed)

seed_everything(42)

In [3]:
# Preprocessing

def preprocess_script(script):
    new_script = deque()
    with open(script,'r',encoding='utf-8') as file:
        lines = file.readlines()
        anotation_checksum = 0
        for line in lines:
            if (line.lstrip().startswith("*/") or line.rstrip().endswith("*/")):
                anotation_checksum = 0
                continue
            if anotation_checksum == 1:
                continue
            if line.lstrip().startswith('#include'): # #include으로 시작되는 행 삭제함
                continue
            if line.lstrip().startswith('/*'): # 주석시작
                if "*/" in line:
                    continue
                else:
                    anotation_checksum = 1 
                    continue

            if line.lstrip().startswith('//'): # 주석으로 시작되는 행 삭제함
                continue
            line = line.rstrip()
            if '//' in line:
                line = line[:line.index('//')] # 주석 전까지 코드만 저장함
            line = line.replace('\n','') # 개행 문자를 모두 삭제함
            line = line.replace('    ','\t') # 공백 4칸을 tab으로 변환함
            
            if line.lstrip().rstrip() == '': # 전처리 후 빈 라인은 삭제함
                continue
            
            new_script.append(line)
            
        new_script = '\n'.join(new_script) # 개행 문자로 합침
        new_script = re.sub('("""[\w\W]*?""")', '<str>', new_script)
        new_script = re.sub("('''[\w\W]*?''')", '<str>', new_script)
        new_script = re.sub('/^(http?|https?):\/\/([a-z0-9-]+\.)+[a-z0-9]{2,4}.*$/', '', new_script)
    
    return new_script

In [4]:
code_folder = "/home/workspace/DACON/CodeSim/Dataset/train_code"
problem_folders = os.listdir(code_folder) 

In [5]:
preprocess_scripts = []
problem_nums = []
problem_idx = []

for problem_folder in tqdm(problem_folders):
    scripts = os.listdir(os.path.join(code_folder, problem_folder)) # code/problem000/
    problem_num = problem_folder # 문제 번호 폴더명
    for script in scripts:
        script_file = os.path.join(code_folder,problem_folder,script) # code/problem000/problem001_000.cpp
        preprocessed_script = preprocess_script(script_file)
        preprocess_scripts.append(preprocessed_script)
        problem_idx.append(script)
    # 번호 목록을 만들어서 전처리한 dataframe에 함께 넣어줌
    problem_nums.extend([problem_num]*len(scripts))

100%|██████████| 500/500 [00:41<00:00, 11.93it/s]


In [6]:
df = pd.DataFrame(data= {'code':preprocess_scripts, 'problem_num':problem_nums, 'problem_idx':problem_idx})

In [7]:
# AutoTokenizer로 graphcodebert 사용하도록 설정
tokenizer = AutoTokenizer.from_pretrained("microsoft/graphcodebert-base")
tokenizer.truncation_side = 'left'
MAX_LEN = 512

In [8]:
tokens = []
for code in tqdm(df['code']):
    tokens.append(tokenizer.tokenize(code, max_length=MAX_LEN, truncation=True))

df['tokens'] = tokens # Sample code를 Tokenization해서 tokens 컬럼에 추가
df['len'] = df['tokens'].apply(len) # tokens의 길이를 len 컬럼에 추가

100%|██████████| 250000/250000 [02:00<00:00, 2071.07it/s]


In [135]:
# train과 validation data set 분리
train_df, valid_df, train_label, valid_label = train_test_split(
        df,
        df['problem_num'],
        random_state=42,
        test_size=0.1,
        stratify=df['problem_num']
    )

train_df = train_df.reset_index(drop=True) # Reindexing
valid_df = valid_df.reset_index(drop=True)

In [136]:
codes = train_df['code'].to_list() # code 컬럼을 list로 변환 - codes는 code가 쭉 나열된 형태임
problems = train_df['problem_num'].unique().tolist() # 문제 번호를 중복을 제외하고 list로 변환
problems.sort()

In [137]:
total_positive_pairs = []
total_negative_pairs = []

In [138]:
for problem in tqdm(problems):
    # 각각의 문제에 대한 code를 골라 정답 코드로 저장, 아닌 문제는 other_codes로 저장
    # 이때 train_df에는 problem_num이 정렬된 상태가 아니기 때문에 index가 다를 수 있음
    solution_codes = train_df[train_df['problem_num'] == problem]['code'].to_list()
    other_codes = train_df[train_df['problem_num'] != problem]['code'].to_list()
    
    # positive_pairs 1000개 (총 500 * 1000 = 500,000개) 추출
    # negative_pairs 1000개 (총 500 * 1000 = 500,000개) 추출
    positive_pairs = list(combinations(solution_codes,2))
    random.shuffle(positive_pairs)
    positive_pairs = positive_pairs[:500]
    random.shuffle(other_codes)
    other_codes = other_codes[:500]
    
    negative_pairs = []
    for pos_codes, others in zip(positive_pairs, other_codes):
        negative_pairs.append((pos_codes[0], others))
    
    total_positive_pairs.extend(positive_pairs)
    total_negative_pairs.extend(negative_pairs)

100%|██████████| 500/500 [01:41<00:00,  4.91it/s]


In [139]:
# total_positive_pairs와 negative_pairs의 정답 코드를 묶어 code1로 지정
# total_positive_pairs와 negative_pairs의 비교 대상 코드를 묶어 code2로 지정
# 해당 코드에 맞는 label 설정
code1 = [code[0] for code in total_positive_pairs] + [code[0] for code in total_negative_pairs]
code2 = [code[1] for code in total_positive_pairs] + [code[1] for code in total_negative_pairs]
label = [1]*len(total_positive_pairs) + [0]*len(total_negative_pairs)

# DataFrame으로 선언
train_data = pd.DataFrame(data={'code1':code1, 'code2':code2, 'similar':label})
train_data = train_data.sample(frac=1).reset_index(drop=True) # frac: 추출할 표본 비율
train_data.to_csv('/home/workspace/DACON/CodeSim/Dataset/train_data.csv',index=False)

In [140]:
codes = valid_df['code'].to_list() # code 컬럼을 list로 변환 - codes는 code가 쭉 나열된 형태임
problems = valid_df['problem_num'].unique().tolist() # 문제 번호를 중복을 제외하고 list로 변환
problems.sort()

In [141]:
total_positive_pairs = []
total_negative_pairs = []

In [142]:
for problem in tqdm(problems):
    # 각각의 문제에 대한 code를 골라 정답 코드로 저장, 아닌 문제는 other_codes로 저장
    # 이때 train_df에는 problem_num이 정렬된 상태가 아니기 때문에 index가 다를 수 있음
    solution_codes = valid_df[valid_df['problem_num'] == problem]['code'].to_list()
    other_codes = valid_df[valid_df['problem_num'] != problem]['code'].to_list()
    
    # positive_pairs 100개 (총 300 * 100 = 30,000개) 추출
    # negative_pairs 100개 (총 300 * 100 = 30,000개) 추출
    positive_pairs = list(combinations(solution_codes,2))
    random.shuffle(positive_pairs)
    positive_pairs = positive_pairs[:100]
    random.shuffle(other_codes)
    other_codes = other_codes[:100]
    
    negative_pairs = []
    for pos_codes, others in zip(positive_pairs, other_codes):
        negative_pairs.append((pos_codes[0], others))
    
    total_positive_pairs.extend(positive_pairs)
    total_negative_pairs.extend(negative_pairs)

100%|██████████| 500/500 [00:07<00:00, 70.67it/s]


In [143]:
# total_positive_pairs와 negative_pairs의 정답 코드를 묶어 code1로 지정
# total_positive_pairs와 negative_pairs의 비교 대상 코드를 묶어 code2로 지정
# 해당 코드에 맞는 label 설정
code1 = [code[0] for code in total_positive_pairs] + [code[0] for code in total_negative_pairs]
code2 = [code[1] for code in total_positive_pairs] + [code[1] for code in total_negative_pairs]
label = [1]*len(total_positive_pairs) + [0]*len(total_negative_pairs)

# DataFrame으로 선언
valid_data = pd.DataFrame(data={'code1':code1, 'code2':code2, 'similar':label})
valid_data = valid_data.sample(frac=1).reset_index(drop=True) # frac: 추출할 표본 비율
valid_data.to_csv('/home/workspace/DACON/CodeSim/Dataset/valid_data.csv',index=False)